In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/turkish-sales-comments/hepsiburada.csv


In [7]:
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM,  Dropout
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [8]:
data = pd.read_csv('../input/turkish-sales-comments/hepsiburada.csv')
df = data.copy()
df.head()

,Rating,Review
0,1,3 yıldır tık demedi. :)
1,1,3 yıldır kullanıyorum müthiş
2,1,Ürün bugün elime geçti çok fazla inceleme fırs...
3,1,Almaya karar verdim. Hemencecik geldi. Keyifle...
4,1,Günlük kullanımınızı çok çok iyi karsılıyor kı...


In [9]:
#0->negatif veri etiketi
#1->pozitif veri etiketi
df['Rating'].unique().tolist()

[1, 0]

In [10]:
# bütün verileri ve etiketleri listeye çeviriyoruz
target = df['Rating'].values.tolist()#negatif=0, pozitif=1
data = df['Review'].values.tolist()#text verisi

In [11]:
#veriyi test ve train verisi olarak ayırıyoruz
#datada veri var targetta etiket var. 
seperation = int(len(data) * 0.80)
x_train, x_test = data[:seperation], data[seperation:]
y_train, y_test = target[:seperation], target[seperation:]

In [12]:

#veri satır ve sütun sayısı
df.shape


(243497, 2)

In [13]:
# Verisetimizde en sık geçen 10000 kelimeyi alıyoruz
num_words = 10000

# Keras ile tokenizer tanımlıyoruz
tokenizer = Tokenizer(num_words=num_words)

In [14]:
# Tüm Veriyi tokenlara ayırıyoruz
tokenizer.fit_on_texts(data)

In [15]:
#tokenizer.word_index #kelimelerin tokenlaşmış halini görebilriizi.

In [16]:
# Tokenizerı kaydediyoruz
import pickle

with open('turkish_tokenizer_hack.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:

# Tokenizerı yüklüyoruz
with open('turkish_tokenizer_hack.pickle', 'rb') as handle:
    turkish_tokenizer = pickle.load(handle)

biz modelimize kelimeleri train olarak vericez ama oradaki kelimeler string halinde tokenlaşmadılar bu yüzden onlarıda tokenlaştırıyoruz.

In [18]:
# Train verisi olarak ayırdığımız veriyi tokenizer ile tokenize ediyoruz

x_train_tokens = turkish_tokenizer.texts_to_sequences(x_train)

In [19]:
x_train[100]


'Bu fiyata bu kalite kaçırmayın derim '

In [20]:
#yukarıdaki x_train[100] olan metni tokenlaştırdıkdan sonra aşağıdaki gibi rakamlarla temsil ediyoruz.
x_train_tokens[100]
#yanlış yazılmış kelimeler olabilir bunların token listemizde olması çok düşük 1 tane falan olur ama biz zaten modeli eğtiriken
#ilk 10000 kelimeeyi tokenlaştırıyrouz. o yüzden etkisi olmuyor çok.

[5, 39, 5, 131, 323, 143]

In [21]:
# Test verisi olarak ayırdığımız veriyi tokenizer ile tokenize ediyoruz

x_test_tokens = turkish_tokenizer.texts_to_sequences(x_test)

In [22]:
#biz modelimize belirli inputlar veririz sabit boyutta bu yüzden bazı inputların size ı 5 ken bazıları 50 olabilir bunu standart haline
#getirmek için az olanlara padding ekliyoruz ve 0 lar ile dolduryoruz.

#Text verileri için padding yapıyoruz
#RNN ağlarını kullanırken önceden belirdiğimiz sabit bir size olur. Tüm input textlerinin sizelarını bu sabit size için padding yaparak 0 lar
#ile doldururuz.

num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)
print(np.mean(num_tokens))
num_tokens.shape

20.744703220162876


(243497,)

In [23]:
# Bütün text verileri içinde maximum token sayısına sahip olanı buluyoruz
max_tokens = np.mean(num_tokens) + 2*np.std(num_tokens) 
max_tokens = int(max_tokens)
max_tokens

59

In [24]:
# Bütün verilere padding yapıyoruz ve bütün veriler aynı boyutta oluyor
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)

In [25]:
# size
print(x_train_pad.shape)
print(x_test_pad.shape)

(194797, 59)
(48700, 59)


In [26]:
model = Sequential() # Kullanacağımız Keras modelini tanımlıyoruz


In [27]:
embedding_size = 50 # Her kelime için vektör boyutunu 50 olarak belirledik


In [28]:
#kelime vektörleri başlangıçta rastgele başlayacak ancak modeli eğitirken kelime vektörlerinide eğiticez. bu şeklide yaptığımızda kelime 
#vektörlerini word3wec kadar olmasada güzel bir sonunç alıcaz

#Kerasta bir embedding layer oluşturuyoruz ve rastgele vektörler oluşturuyoruz

# Modele embedding layer ekliyoruz
# embedding matris size = num_words * embedding_size -> 10.000 * 50
#10.000 e 50 bir matris oluşturuyoruz.
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))

#bu layer input aldığı vektörleri bir sonraki layera gönderiyor. 
#örnek olarak "bu ürün süper" yorumun geldi 10k kelimenin arasından bu kelimelere karşılık gelen vektörleri alıcaz bir sonraki layera vericez

In [29]:
# 3-katmanlı(layer) LSTM
model.add(LSTM(units=16, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=8, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=4, return_sequences=False)) # bir sonraki layer lstm olmadığı için false
model.add(Dropout(0.2))
# Dense layer: Tek nörondan oluşuyor ve değerler tek bir değer olarak sigmoide verilir
model.add(Dense(1, activation='sigmoid'))# Sigmoid aktivasyon fonksiyonu

In [30]:
# Adam optimizer
from keras.optimizers import Adam
optimizer = Adam(lr=1e-3)

In [31]:
# Farklı optimizerları deniyoruz
model.compile(loss='binary_crossentropy',
optimizer='adam',
metrics=['accuracy'])

In [32]:
# modelin özeti
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 59, 50)            500000    
_________________________________________________________________
lstm (LSTM)                  (None, 59, 16)            4288      
_________________________________________________________________
dropout (Dropout)            (None, 59, 16)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 59, 8)             800       
_________________________________________________________________
dropout_1 (Dropout)          (None, 59, 8)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4)                 208       
_________________________________________________________________
dropout_2 (Dropout)          (None, 4)                 0

In [34]:
# epoch -> veri ile kaç kere eğiteceğiz
# batch_size -> feeding size-her epochta kaç veri ile besleyeceğiz
model.fit(np.array(x_train_pad), np.array(y_train), epochs=5, batch_size=256)


Epoch 1/5
761/761 [==============================] - 23s 17ms/step - loss: 0.3739 - accuracy: 0.9087
Epoch 2/5
761/761 [==============================] - 12s 16ms/step - loss: 0.1943 - accuracy: 0.9472
Epoch 3/5
761/761 [==============================] - 12s 15ms/step - loss: 0.1041 - accuracy: 0.9674
Epoch 4/5
761/761 [==============================] - 12s 16ms/step - loss: 0.0770 - accuracy: 0.9774
Epoch 5/5
761/761 [==============================] - 12s 16ms/step - loss: 0.0607 - accuracy: 0.9830


In [36]:
# model sonuçları
result = model.evaluate(np.array(x_test_pad), np.array(y_test))
result

1522/1522 [==============================] - 8s 5ms/step - loss: 0.1441 - accuracy: 0.9525


[0.144137442111969, 0.9525461792945862]

In [37]:

# doğruluk oranı
accuracy = (result[1]) * 100
accuracy

95.25461792945862

In [38]:
#test yorumları(inputlar)
text1 = "böyle bir şeyi kabul edemem"
text2 = "tasarımı güzel ancak ürün açılmış tavsiye etmem"
text3 = "bu işten çok sıkıldım artık"
text4 = "kötü yorumlar gözümü korkutmuştu ancak hiçbir sorun yaşamadım teşekkürler"
text5 = "yaptığın işleri hiç beğenmiyorum"
text6 = "tam bir fiyat performans ürünü beğendim"
text7 = "Bu ürünü beğenmedim"
texts = [text1, text2,text3,text4,text5,text6,text7]

In [39]:
tokens = turkish_tokenizer.texts_to_sequences(texts)
tokens = turkish_tokenizer.texts_to_sequences(texts)
tokens

[[200, 2, 1231, 2558],
 [553, 7, 82, 4, 5984, 9, 1031],
 [5, 7293, 1, 214],
 [177, 735, 7728, 82, 263, 105, 326, 16],
 [3276, 46],
 [74, 2, 28, 111, 19, 146],
 [5, 19, 1365]]

In [40]:
#padding
tokens_pad = pad_sequences(tokens, maxlen=max_tokens)

In [41]:
#model bu yorumların hangi duyguya yakın olduğunu tahminliyor
model.predict(tokens_pad)

array([[0.35313267],
       [0.24135295],
       [0.9894964 ],
       [0.9956045 ],
       [0.923757  ],
       [0.9981462 ],
       [0.10886286]], dtype=float32)

In [43]:
for i in model.predict(tokens_pad):
    if i < 0.5:
        print("negatif")#negatif yorum yapmış
    else:
        print("pozitif")#pozitif yorum yapmış

negatif
negatif
pozitif
pozitif
pozitif
pozitif
negatif


In [44]:

from keras.models import load_model

model.save('model_lstm.h5')  # modeli kaydediyoruz